In [30]:
import os
import json
from PIL import Image

root_dir = '/standard/UVA-DSA/NIST EMS Project Data/EgoExoEMS_CVPR2025/Dataset/Final/'

In [33]:

def count_num_masks(masks_dir):
    mask_stats = {}

    for file_name in os.listdir(masks_dir):
        if file_name.startswith("mask_") and file_name.endswith(".png"):
            # Extract the object type from the file name
            try:
                _, frame_number, object_type = file_name.split("_")
                object_type = object_type.split(".")[0]  # Remove ".png"

                if object_type == "4":
                    continue

                # Open the image to check if it contains any non-zero pixels
                image_path = os.path.join(masks_dir, file_name)
                with Image.open(image_path) as img:
                    # Convert to grayscale and check for non-zero pixels
                    if img.convert("L").getextrema()[1] > 0:  # Max pixel > 0 indicates a non-empty mask
                        # Update count for the object type
                        if object_type in mask_stats:
                            mask_stats[object_type] += 1
                        else:
                            mask_stats[object_type] = 1
            except ValueError:
                print(f"[WARNING] File name {file_name} does not match the expected pattern.")
            except Exception as e:
                print(f"[ERROR] Could not process {file_name}: {e}")

    return mask_stats


In [36]:
import os
import json

# Set to keep track of processed trials
processed_trials = set()
total_segmask_stats = {}

# Iterate recursively through all files in the root directory
for root, dirs, files in os.walk(root_dir):
    if "segmentations" in root:
        trial_path = "/".join(root.split("/")[:-2])
        
        # Skip this trial if already processed
        if trial_path in processed_trials:
            continue

        print(f"[INFO] Processing segmentation masks: {root}")
        print(f"[INFO] Trial path: {trial_path}")

        subject_id = trial_path.split("/")[-3]
        trial_id = trial_path.split("/")[-1]
        print(f"[INFO] Subject ID: {subject_id}")
        print(f"[INFO] Trial ID: {trial_id}")

        # Count the segmentation masks in the current directory
        mask_stats = count_num_masks(root)

        # Store the results in the total_segmask_stats dictionary with subject->trial hierarchy
        if subject_id not in total_segmask_stats:
            total_segmask_stats[subject_id] = {}
        total_segmask_stats[subject_id][trial_id] = mask_stats

        # Mark this trial as processed to avoid re-processing
        processed_trials.add(trial_path)

        print(f"[INFO] Segmentation mask statistics: {mask_stats}")
        print("*" * 10, "=" * 10, "*" * 10)

# Final segmentation mask statistics for all processed trials
print("[INFO] Total segmentation mask statistics:", total_segmask_stats)


[INFO] Processing segmentation masks: /standard/UVA-DSA/NIST EMS Project Data/EgoExoEMS_CVPR2025/Dataset/Final/ng2/cardiac_arrest/1/BBOX_MASKS/segmentations
[INFO] Trial path: /standard/UVA-DSA/NIST EMS Project Data/EgoExoEMS_CVPR2025/Dataset/Final/ng2/cardiac_arrest/1
[INFO] Subject ID: ng2
[INFO] Trial ID: 1
[INFO] Segmentation mask statistics: {'1': 162, '3': 6}
********** ========== **********
[INFO] Processing segmentation masks: /standard/UVA-DSA/NIST EMS Project Data/EgoExoEMS_CVPR2025/Dataset/Final/ng2/cardiac_arrest/0/BBOX_MASKS/segmentations
[INFO] Trial path: /standard/UVA-DSA/NIST EMS Project Data/EgoExoEMS_CVPR2025/Dataset/Final/ng2/cardiac_arrest/0
[INFO] Subject ID: ng2
[INFO] Trial ID: 0
[INFO] Segmentation mask statistics: {'1': 155, '3': 28}
********** ========== **********
[INFO] Processing segmentation masks: /standard/UVA-DSA/NIST EMS Project Data/EgoExoEMS_CVPR2025/Dataset/Final/ms2/cardiac_arrest/3/BBOX_MASKS/segmentations
[INFO] Trial path: /standard/UVA-DSA/NIS

In [39]:
# Initialize variables to store the total count of bounding boxes and segmentation masks
total_masks_count = 0

# Calculate total count of segmentation masks
for subject_id, trials in total_segmask_stats.items():
    for trial_id, mask_stats in trials.items():
        total_masks_count += sum(mask_stats.values())

print(f"[INFO] Total number of segmentation masks across all subjects: {total_masks_count}")


[INFO] Total number of segmentation masks across all subjects: 12045
